# SQL ON SHELL

**SERHAT ÇEVİKEL**

First let's start our PostgreSQL server again:

In [ ]:
/usr/lib/postgresql/11/bin/pg_ctl start -D /etc/postgresql/11/main/ -m smart

In order to check whether the server is responding by:

In [ ]:
psql -U postgres -c "\l"

Now let's create our environment variables and extract the zipped tsv files if they are not yet:

In [ ]:
datadir=~/data
imdbdir=$datadir/imdb

In [ ]:
mkdir -p $imdbdir/tsv2

In [ ]:
find $imdbdir/tsv -mindepth 1 | \
    parallel -k -j0 "basenm=\$(basename {});
        if [ ! -e ${imdbdir}/tsv2/\${basenm%.gz} ];
        then
            gunzip -c {} > \
            ${imdbdir}/tsv2/\${basenm%.gz};
        fi
        "

Now we will go through two similar shell tools to use sql syntax on csv or tsv files:

## Q

The source page for q project is:

https://github.com/harelba/q

```
q - Text as Data

q is a command line tool that allows direct execution of SQL-like queries on CSVs/TSVs (and any other tabular text files).

q treats ordinary files as database tables, and supports all SQL constructs, such as WHERE, GROUP BY, JOINs etc. It supports automatic column name and column type detection, and provides full support for multiple encodings.

q's web site is http://harelba.github.io/q/. It contains everything you need to download and use q in no time.
```

The tsv files are as follows:

In [ ]:
tldr q

In [ ]:
ls $imdbdir/tsv2

A very basic query taking stdin as input: 

In [ ]:
cat $imdbdir/tsv2/title.ratings.tsv | \
q -t -H "SELECT * \
FROM - \
WHERE tconst = 'tt0000001';"

Or use the filename inside FROM clause:

In [ ]:
q -t -H "SELECT * \
FROM $imdbdir/tsv2/title.ratings.tsv \
WHERE tconst = 'tt0000001';"

## CSVSQL

Part of the csvkit suite:

https://github.com/wireservice/csvkit

https://csvkit.readthedocs.io/en/1.0.3/

In [ ]:
cat $imdbdir/tsv2/title.ratings.tsv | csvsql -t --query "SELECT * \
FROM stdin \
WHERE tconst = 'tt0000001';"

# COMMON TABLE EXPRESSIONS IN SQL (WITH QUERIES)

In order to use sql magic on Python3 kernel:

In [ ]:
%load_ext sql

In [ ]:
%sql postgres://postgres@localhost/imdb2

You can remember the structure of the data by either typing:

In [ ]:
psql -U postgres -d imdb2 -c "\d+ public.*"

Or viewing the file:

In [ ]:
cat ~/imdb_database

## Godfather cast with CTE's

Now let's remember the query from session 2:


- Filter for movies titled Godfather.\\*Part
- Genres include drama and exclude comedy
- Start year not after 1990
- People with primary profession actor|actress

- First join titles and principal cast on title id's (tconst)
- And then join principal cast and name basics on name id's (nconst)
- Filter only for actors and actresses
- And sort on first names (ascending) then title years

In [ ]:
%%sql

SELECT tb.tconst, tb.originaltitle, tb.startyear, tb.runtimeminutes, tb.genres,
    tp.principalcast,
    nb.primaryname, nb.birthyear, nb.deathyear, nb.primaryprofession

FROM title_basics tb
LEFT JOIN title_principals_melt tp USING (tconst)
	LEFT JOIN name_basics nb ON tp.principalcast=nb.nconst

WHERE tb.originaltitle ~ 'Godfather.*Part'
	AND tb.genres ~ '(?i)drama'
	AND NOT tb.genres ~ '(?i)comedy'
	AND tb.startyear <= 1990
	AND nb.primaryprofession ~'actor|actress'

ORDER BY nb.primaryname, tb.startyear DESC;

We can design this query so that smaller steps are combined:

- Now let's rewrite the previous query with common table expressions (CTE) (aka "WITH" queries)
- With CTE, we run a subquery or multiple subqueries before the main query, name it/them, refer to other subqueries with those names
- And reuse the subqueries in multiple places in the main query - referring by its name, since PostgreSQL stores the results of subqueries
- Just like a table!

In [ ]:
%%sql

WITH basics_principles AS
(
SELECT tb.tconst, tb.originaltitle, tb.startyear, tb.runtimeminutes,
	tb.genres, tp.principalcast
FROM title_basics tb
    LEFT JOIN title_principals_melt tp
    USING (tconst)
WHERE tb.originaltitle ~ '.*Godfather.*Part.*'
	AND tb.genres ~ '(?i)drama'
	AND NOT tb.genres ~ '(?i)comedy'
	AND tb.startyear <= 1990
)
SELECT bp.tconst, bp.originaltitle, bp.startyear, bp.runtimeminutes,
	bp.genres, bp.principalcast, nb.primaryname,
	nb.birthyear, nb.deathyear, nb.primaryprofession

FROM basics_principles bp LEFT JOIN name_basics nb ON bp.principalcast=nb.nconst

WHERE nb.primaryprofession ~'actor|actress'

ORDER BY nb.primaryname, bp.startyear DESC;

## De Niro and Scorsese

This was already easily implemented without CTE's.

Now a harder example which is much challenging to implement without CTE's:

Return all fields from title_basics for "movies" which feature Martin Scorsese as director and Robert De Niro as actor, sorted by ascending startyear

In [ ]:
%%sql

-- first get the nconst of Martin Scorsese
WITH ms_id AS
(
SELECT nb.nconst
FROM name_basics nb
WHERE nb.primaryname = 'Martin Scorsese'
	AND nb.primaryprofession ~ 'director'
),
-- then get the titles directed by Martin Scorsese
ms_titles AS
(
SELECT tc.tconst
FROM ms_id LEFT JOIN title_crew tc ON ms_id.nconst=tc.directors
),
-- and get the nconst of Robert De Niro
rdn_id AS
(
SELECT nb.nconst
FROM name_basics nb
WHERE nb.primaryname = 'Robert De Niro'
),
-- and get the titles by Robert De Niro
rdn_titles AS
(
SELECT tp.tconst
FROM rdn_id LEFT JOIN title_principals_melt tp ON rdn_id.nconst=tp.principalcast
)
-- and last, get the intersection of titles by De Niro and Scorsese
SELECT tb.*
FROM ms_titles INNER JOIN rdn_titles ON ms_titles.tconst=rdn_titles.tconst
LEFT JOIN title_basics tb ON rdn_titles.tconst=tb.tconst
WHERE tb.titletype = 'movie'
ORDER BY tb.startyear


## Ratings of Al Pacino

Now another:


- Return all the fields from title_basics and "averagerating" field from title_ratings of the "movies" that Al Pacino took role in
- Exclude titles which have no "averagerating" info (with "IS NOT NULL" logical test)
- Order by descending averageratings
- Note that: In order to get the id of Al Pacino, from the name_basics table, you should check the "primaryname" field for equality with "Al Pacino" and birthyear with "1940"  
- You can use multiple "common table expressions" (CTE) or "WITH" queries to simplify the main query
- You can use aliases for tables to simplify your query (e.g. FROM title_basics tb)

In [ ]:
%%sql

-- get the nconst of pacino
WITH pacino AS
(
SELECT nconst
FROM name_basics nb
WHERE nb.primaryname = 'Al Pacino'
	AND nb.birthyear = 1940
),
-- get the titles of pacino
t_ids AS
(
SELECT tp.tconst
FROM pacino LEFT JOIN title_principals_melt tp ON pacino.nconst=tp.principalcast
)
-- get the details of those titles
SELECT tb.*, tr.averagerating
FROM t_ids LEFT JOIN title_basics tb ON t_ids.tconst=tb.tconst
	LEFT JOIN title_ratings tr ON tb.tconst=tr.tconst
WHERE tb.titletype = 'movie'
	AND tr.averagerating IS NOT NULL
ORDER BY tr.averagerating DESC	

## Youngest cast of each movie, revisited

**EXERCISE 1:**

Now remember the last query from session 3, in which we tried to get the oldest cast of each filtered movie.

With CTE's we can do it in a more efficient and neat way:


- Select movies with averagerating > 8.8, numvotes > 500000, titletype is not tvSeries
- Now calculate the max birthyear of the cast of each movies and select those names with max birthyear, hence youngest cast
- Report primarytitle, primaryname, birthyear of cast, startyear of movie and age of the youngest cast at the time of the movie (just the difference of previous two columns aliased as "age"

The version with subquery was as such:


```SQL
SELECT mb.primarytitle, nb.birthyear, tb.startyear, tb.startyear - nb.birthyear as age,nb.primaryname
    FROM
(
SELECT tb.primarytitle, max(nb.birthyear) as maxbirthyear, tb.tconst 
FROM title_basics tb
    LEFT JOIN title_ratings tr USING (tconst)
    LEFT JOIN title_principals_melt tp USING (tconst)
    LEFT JOIN name_basics nb ON nb.nconst=tp.principalcast
WHERE tr.averagerating > 8.8
    AND tr.numvotes > 500000
    AND NOT tb.titletype = 'tvSeries'
GROUP BY tb.primarytitle, tb.tconst
) mb
    LEFT JOIN title_basics tb USING (tconst)
    LEFT JOIN title_principals_melt tp USING (tconst)
    LEFT JOIN name_basics nb ON nb.nconst=tp.principalcast
WHERE nb.birthyear = mb.maxbirthyear
```

and the basic query listing the primary casts was as such:

```SQL
SELECT tb.primarytitle, tb.startyear, nb.primaryname, nb.birthyear, tr.averagerating
FROM title_basics tb
    LEFT JOIN title_ratings tr USING (tconst)
    LEFT JOIN title_principals_melt tp USING (tconst)
    LEFT JOIN name_basics nb ON nb.nconst=tp.principalcast
WHERE tr.averagerating > 8.8
    AND tr.numvotes > 500000
    AND NOT tb.titletype = 'tvSeries'
ORDER BY nb.primaryname, tb.primarytitle;
```

In [ ]:
%%sql

-- statement here

In [ ]:
psql -U postgres -d imdb2 <<EOF

-- or you may write your statement here
EOF

**SOLUTION 1:**

In [ ]:
pass1=
encrypt=$(cat crypt/encrypt_04_01.crypt)
solution=$(echo "$encrypt" | openssl enc -aes-128-cbc -a -d -salt -pass pass:$pass1 2> /dev/null)
echo "$solution"; echo
for l in "$solution"; do eval "${l}"; done

# SQL FROM R

Now we will see how we can access to a PostgreSQL database from R

## Using RPostgreSQL package

First load necessary package

In [ ]:
library(RPostgreSQL)

Load PostgreSQL driver

In [ ]:
drvv <- dbDriver("PostgreSQL")
drvv

Create a connection to a database using the driver

In [ ]:
con <- dbConnect(drvv,
                 dbname = "imdb2",
                 host = "localhost",
                 port = 5432,
                 user = "postgres")

con

Get list of table names

In [ ]:
table_names <- dbGetQuery(con,
                          "SELECT table_name
                          FROM information_schema.tables 
                          WHERE table_type = 'BASE TABLE'
                          AND table_schema = 'public'
                          ORDER BY table_name
                          "
                          )

table_names

class(table_names)

Or more easily with:

In [ ]:
dbListTables(con)

Good! The query returns a data frame

Now we can loop through tables to automatize queries

List tables:

In [ ]:
table_names_vec <- dbListTables(con)
table_names_vec
class(table_names_vec)

This yields a character vector, not a data frame

Get row counts

In [ ]:
# create a query template that we can change the parameters of
# each "%s" stands for a variable that we will manipulate with sprintf
query_text <- "SELECT count (*) FROM %s"

# create an empty vector, the length of the table names
# we will populate this vector with the row counts from tables
row_counts <- rep(NA, length(table_names_vec))

# update the names of the vector with table names
names(row_counts) <- table_names_vec

# for across table names indices (not the names themselves)
for (tbl_ind in seq_along(table_names_vec))
{
    # replace the %s pointer in query text with table name
    current_query <- sprintf(query_text, table_names_vec[tbl_ind])
    
    # run the query and get results
    table_name <- dbGetQuery(con, current_query)
    
    # assign the query result to vector
    row_counts[tbl_ind] <- table_name[[1]]
}
                  
return(row_counts)

### Play with title_ratings

Let's import a table as a data frame into R

In [ ]:
title_ratings_df <- dbGetQuery(con, "SELECT * from title_ratings")

Get the head, attributes and summary of the data frame

In [ ]:
head(title_ratings_df)

In [ ]:
class(title_ratings_df)

Now an easier way to read data into R

In [ ]:
title_ratings_df_b <- dbReadTable(con, "title_ratings")

See whether they are identical objects

In [ ]:
identical(title_ratings_df, title_ratings_df_b)

So, they are identical

What is the object size of each data frame?

In [ ]:
format(object.size(title_ratings_df), units = "auto")

In [ ]:
format(object.size(title_ratings_df_b), units = "auto")

They take too much space. Now let's get rid of one of them

In [ ]:
rm(title_ratings_df_b)

New get the classes of all columns

In [ ]:
str(title_ratings_df)

### Play with title_basics

Let's query title_basics as such:

Let's say, take rows in which 

name contains "Star Wars",

not an adult movie

title_type is a movie and

start year is between 1977 and 2016

In [ ]:
title_basics_df2 <- dbGetQuery(con,
                          "SELECT *
                            FROM title_basics 
                            WHERE primarytitle ~* 'star.*wars'
                            AND isadult = false
                            AND titletype = 'movie'
                            AND startyear BETWEEN 1977 AND 2016"
                          )

In [ ]:
title_basics_df2

In [ ]:
str(title_basics_df2)

Did not like NA's in tconst, let's enhance it

In [ ]:
title_basics_df3 <- title_basics_df2[
    !is.na(title_basics_df2$tconst),
]

In [ ]:
title_basics_df3

Much better!

Now let's write this data frame into our imdb2 database as a new table

In [ ]:
dbWriteTable(con, "star_wars",
             value = title_basics_df3,
             append = TRUE, row.names = FALSE)

Check whether table is created:

In [ ]:
dbExistsTable(con, "star_wars")

And read from the database again

In [ ]:
star_wars <- dbReadTable(con, "star_wars")

star_wars

See, the only difference seems to be the rownames which we excluded on purpose

Apart from that, let's check whet they are both identical

In [ ]:
attributes(title_basics_df3)

In [ ]:
rownames(title_basics_df3) <- NULL
title_basics_df3

In [ ]:
identical(title_basics_df3, star_wars)

Yes they are identical!

### Primary constraints

Let's remove the table

In [ ]:
dbRemoveTable(con, "star_wars")

Check whether it still exists

In [ ]:
dbExistsTable(con, "star_wars")

Now let's create the table again

In [ ]:
dbWriteTable(con, "star_wars",
             value = title_basics_df3, append = TRUE, row.names = FALSE)

Check that it exists:

In [ ]:
dbExistsTable(con, "star_wars")

Get its rowcount:

In [ ]:
dbGetQuery(con, "SELECT count (*) FROM star_wars")

And repeat the data write statement, with append option

In [ ]:
dbWriteTable(con, "star_wars",
             value = title_basics_df3,
             append = TRUE, row.names = FALSE)

Get the rowcount again

In [ ]:
dbGetQuery(con, "SELECT count (*) FROM star_wars")

<a href="https://www.youtube.com/embed/tsXEToflqGs?start=10&autoplay=1">OW! I HAVE A BAD FEELING ABOUT THIS</a>

We did not define a primary key, so postgresql server allowed duplicates to be inserted in to the database

Now delete and recreate table

In [ ]:
dbRemoveTable(con, "star_wars")

In [ ]:
dbExistsTable(con, "star_wars")

In [ ]:
dbWriteTable(con, "star_wars",
             value = title_basics_df3,
             append = TRUE, row.names = FALSE)

In [ ]:
dbExistsTable(con, "star_wars")

And add a primary key constraint

In [ ]:
dbGetQuery(con,           
"
ALTER TABLE star_wars
ADD CONSTRAINT tconst_pk_10 
PRIMARY KEY (tconst);
")

Let's try to import the data again

In [ ]:
dbWriteTable(con, "star_wars",
             value = title_basics_df3,
             append = TRUE, row.names = FALSE)

See that:

***could not Retrieve the result : ERROR:  duplicate key value violates unique constraint "tconst_pk_10***

Let's check the row count again:

In [ ]:
dbGetQuery(con, "SELECT count (*) FROM star_wars")

OK, we duplicates were really not allowed!

Let's close connection

In [ ]:
dbDisconnect(con)

And unload the driver

In [ ]:
dbUnloadDriver(drvv)

## Using sqldf package

First let's load the package:

In [ ]:
library(RPostgreSQL)
library(sqldf)

And set the options:

In [ ]:
options(sqldf.RPostgreSQL.user ="postgres", 
  sqldf.RPostgreSQL.dbname ="imdb2",
  sqldf.RPostgreSQL.host ="localhost", 
  sqldf.RPostgreSQL.port =5432)

### Average runtimes

Now let's run a query to get average runtimeminutes from title_basics and group them and order them by startyear  

In [ ]:
query = "SELECT avg(runtimeminutes) runtimeminutes, startyear 
            FROM title_basics
            GROUP BY startyear
            ORDER BY startyear;"

av_runtime <- sqldf(query)

In [ ]:
av_runtime

class(av_runtime)

It returns a data frame

Now subset years between 1940 and 2016

In [ ]:
av_runtime_subset <- av_runtime[av_runtime$startyear %in% 1940:2016,]

av_runtime_subset

Check whether any NA's exist

In [ ]:
which(is.na(av_runtime_subset[[1]]))

Let's plot the data

In [ ]:
runtime_ts <- ts(av_runtime_subset[[1]],
                 start = av_runtime_subset[1,2],
                 end = av_runtime_subset[nrow(av_runtime_subset),2])

runtime_ts

In [ ]:
plot(runtime_ts)

Well that data may have many different types of titles including shorts, etc

Let's first subset for movies

In [ ]:
query = "SELECT avg(runtimeminutes) runtimeminutes, startyear 
            FROM title_basics
            WHERE titletype = 'movie'
            GROUP BY startyear
            ORDER BY startyear;"

av_runtime <- sqldf(query)

In [ ]:
av_runtime

In [ ]:
av_runtime_subset <- av_runtime[av_runtime$startyear %in% 1940:2016,]

av_runtime_subset

In [ ]:
runtime_ts <- ts(av_runtime_subset[[1]],
                 start = av_runtime_subset[1,2],
                 end = av_runtime_subset[nrow(av_runtime_subset),2])

runtime_ts

In [ ]:
plot(runtime_ts)

### Explore Hitchcock movies

In [ ]:
library(RPostgreSQL)

In [ ]:
drvv <- dbDriver("PostgreSQL")
drvv

In [ ]:
con <- dbConnect(drvv,
                 dbname = "imdb2",
                 host = "localhost",
                 port = 5432,
                 user = "postgres")

con

Now the task is:
- First get the count of average rating of movies by Alfred Hitchcock (excluding NULL ratings) with an SQL query
- And summarize these values by decades (total count of movies and average rating by decade)
- You can use floor division by floor(a/b) or its operator %/% in R
- You can use aggregate function in R
- You can recycle all codes that we have written sor far, or use the net
- You can cooperate

In [ ]:
query <- "
WITH hitch AS
(
SELECT nconst
FROM name_basics nb
WHERE nb.primaryname = 'Alfred Hitchcock'
	AND nb.primaryprofession ~ 'director'
),
t_ids AS
(
SELECT tp.tconst
FROM hitch LEFT JOIN title_crew tp ON hitch.nconst=tp.directors
)
SELECT startyear, count(*), avg(tr.averagerating) avrate
FROM t_ids LEFT JOIN title_basics tb ON t_ids.tconst=tb.tconst
	LEFT JOIN title_ratings tr ON tb.tconst=tr.tconst
WHERE tb.titletype = 'movie'
	AND tr.averagerating IS NOT NULL
GROUP BY startyear
ORDER BY startyear
"

In [ ]:
hitch_df <- dbGetQuery(con, query)

In [ ]:
hitch_df

In order to get an average rating for each decade weighted with the count of movies for each year, it is good to have a column for sum of ratings for a year:

In [ ]:
hitch_df$sumrate <- hitch_df$count * hitch_df$avrate

In [ ]:
hitch_df

Now let's get a column for decades:

In [ ]:
hitch_df$dec <- (hitch_df$startyear %/% 10) * 10

In [ ]:
hitch_df

And let's aggregate for each decade the sums of sumrate values:

In [ ]:
hitch_agg <- aggregate(hitch_df[,c(2,4)],
                       by = list(hitch_df$dec),
                       FUN = sum)

In [ ]:
hitch_agg

And let's get the average rating for each decade, deleting the sumrate column:

In [ ]:
hitch_agg$avrate <- hitch_agg$sumrate / hitch_agg$count

In [ ]:
hitch_agg$sumrate <- NULL

In [ ]:
hitch_agg

Let's have a scatterplot of counts versus average ratings

In [ ]:
plot(hitch_agg[,-1])

Let's have the total counts by decade:

In [ ]:
plot(x = hitch_agg[[1]], y = hitch_agg[[2]], main = "total count by decade")
lines(x = hitch_agg[[1]], y = hitch_agg[[2]], type = "l")

And let's have the averate ratings by decade

In [ ]:
plot(x = hitch_agg[[1]], y = hitch_agg[[3]], main = "av ratings by decade")
lines(x = hitch_agg[[1]], y = hitch_agg[[3]], type = "l")

## Using dbplyr package (optional)

Now let's go through a final example using tidyverse and data.table and an R package called "dbplyr" that incorporates dplyr verbes, converts a dplyr pipe to an sql query and executes remotely.

Apart from dbplyr usage, this example will be a simple showcase of tidyverse and data.table tools

In [ ]:
library(dbplyr)
library(tidyverse)
library(data.table)
library(RPostgreSQL)

### Data connection and querying

In [ ]:
drvv <- dbDriver("PostgreSQL")
drvv

Create a connection to a database using the driver

In [ ]:
con <- dbConnect(drvv,
                 dbname = "imdb2",
                 host = "localhost",
                 port = 5432,
                 user = "postgres")

con

And create a remote source object:

In [ ]:
title_basics <- tbl(con, "title_basics")

In [ ]:
title_basics

Now let's create an sqlquery using dplyr verbes:

- Select genres and startyear columns
- Filter for startyear between 1950-2017, movie titletype and exclude missing genres
- Create a new column "count" for the record count (of any column)
- Group by genres and startyear and return the count of records

In [ ]:
query1 <- title_basics %>% select(c("titletype", "genres", "startyear")) %>%
filter(between(startyear, 1950, 2017) &
       !is.na(genres) &
       titletype == "movie") %>%
group_by(genres, startyear) %>%
summarise(count = n())

This is lazy query it is not executed and returned yet:

In [ ]:
query1

Show the resulting sql query:

In [ ]:
query1 %>%show_query

Execute and collect the data:

In [ ]:
table1 <- query1 %>% collect

### Data exploration and wrangling

Summarize the data:

In [ ]:
str(table1)

In [ ]:
table1 %>% glimpse

Get unique values:

In [ ]:
table1 %>% select(-count) %>% sapply(unique)

Any missing values?

In [ ]:
which(!complete.cases(table1))

Convert to a data.table object:

In [ ]:
setDT(table1)

Now what we will do with this data is that we will explore the count of multiple genres in titles:

Let's create a new column that shows how many genres each movie has:

In [ ]:
table1[,genrec := stringr::str_extract_all(genres, ",", simplify= T) %>% length %>% "+"(1),
       by = 1:nrow(table1)]

See how "genres" and "genrec" columns are related:

In [ ]:
table1[,.SD[1], by = genres]

Now let's create contingency tables of genre counts as proportions for each startyear:

In [ ]:
table2 <- table1[,.(countx = sum(count)), by = c("startyear", "genrec")]

In [ ]:
table3 <- table2[, .(countr = countx / sum(countx), genrec), by = "startyear"]

In [ ]:
genre_counts <- table1[, as.list(table(genrec) %>% prop.table %>% "*"(100) %>% round(2)),
                 by = startyear]
genre_counts

However, in time, more genre combinations inflate the total count of those unique combinations. We should also aggregate the movie counts in these genre combinations as such:

In [ ]:
table2 <- table1[,.(countx = sum(count)), by = c("startyear", "genrec")]
table3 <- table2[, .(countr = countx / sum(countx), genrec), by = "startyear"]

table3 %>% spread("genrec", "countr")

### Data visualization

Let's visualize the proportions of 1, 2, and 3 genred movies (first the count of unique genre combinations without weighting with movie counts) as a stacked area chart:

In [ ]:
plot1 <- genre_counts %>%
    gather("key", "value", -startyear) %>%
    ggplot(aes(x = startyear, y = value, fill = key)) +
        geom_area()
        
plot1

We can also visualize interactively using plotly:

In [ ]:
plot1 %>% plotly::ggplotly()

See that over time movies represent multiple genres as opposed to single genres to a larger extent (increase in the area of "3").

However we should also try to visualize the summary data in which the count of movies in each unique genre combination is also used for weighting:

In [ ]:
plot2 <- table3 %>%
    ggplot(aes(x = startyear, y = countr, fill = as.factor(genrec))) +
        geom_area()

plot2

We can also visualize interactively using plotly:

In [ ]:
plot2 %>% plotly::ggplotly()

Here, we see that, in fact the expansion of the share of movies with 3 categories is only due to more unique genre combinations over time. However total counts of moviesare dispersed across more categories. So the weightings with movie counts show that movies with single genres still dominate